In [4]:
!pip install simpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#with interrupt handling - capacity 20 - preemption
import random
import simpy

RANDOM_SEED = 42  
NEW_CUSTOMERS = 5  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience
NO_OF_COUNTERS = 0     #variable that keeps track of number of counters
COUNT_CUSTOMER = 0     #variable that keeps track of number of customers

class Counter(object):
    def __init__(self, env):       #Constructor
        global NO_OF_COUNTERS       #declared global before using global variables orelse python will treat it as local variable
        NO_OF_COUNTERS += 1         #increased by 1 for each object calls. Since called once NO_OF_COUNTERS becomes 1 from 0
        self.env = env              #this particular class instance's local env (environment) = assigned env passed in param
        self.new_counter = simpy.PreemptiveResource( self.env, capacity = 20 )  #PreemptiveResource with capacity 20 for our environment called from simpy
        print( '%7.4f Counter%02d: Opened' % (self.env.now, NO_OF_COUNTERS) )     #prints counter open message at the beginning 


def generate_customer(env, number, interval, counter):    
    global COUNT_CUSTOMER
    global NO_OF_COUNTERS
    for i in range(number):        #loops upto NEW_CUSTOMERS (total number of customers allowed in this simulation)
        COUNT_CUSTOMER += 1        #update the customer count by 1 -> creating each new customer with each loop iteration
        if (COUNT_CUSTOMER > 5):    #Condition checks -> if more than 5 customers waiting in line unattended as when each 
                                    #customer is tend to, they leave decreasing customer count by 1 
          NO_OF_COUNTERS += 1       #increases number of counters by 1 indicating new counter has been opened
          print("%7.4f: More than 5 customers waiting in line" % env.now)
          print( '%7.4f Counter%02d: Opened' % ( env.now, NO_OF_COUNTERS ) )  #prints the environment time at the moment (0.0000 format) and 
                                                                              #the opened counter number (00 format)
          COUNT_CUSTOMER = 1     #COUNT_CUSTOMER set to 1 because when new counter opens we have to again keep track if this phenomena repeats in the future and handle it
        c = Customer(env, COUNT_CUSTOMER, 'Customer%02d' % i, counter, time_in_bank=12.0)   #a instance/ object of Customer class created -> indicates creation of each customer 
        t = random.expovariate(1.0 / interval)    #selects a random value from the interval (1/INTERVAL_CUSTOMERS) -> (1/10.0) here
        yield env.timeout(t)             #env times out after 't' time -> after 't' time another customer creation process starts


class Customer(object):  
    def __init__(self, env, count_cus, name, counter, time_in_bank):
          self.env = env                   #env, COUNT_CUSTOMER, customer number, instance of counter and time to be spent in bank 
          self.count_cus = count_cus       #all passed to each customer instance as "self.___" from the respective param initializations 
          self.name = name
          self.counter = counter
          self.time_in_bank = time_in_bank        #a considered value for Time to be spend in bank for a customer to complete business generally  
          self.process = env.process(self.customer())     #Starting process -> calling customer() method -> here for this particular instance 

    def customer(self): 
        global NO_OF_COUNTERS
        arrive = self.env.now       
        print( '%7.4f %s: Here I am' % (arrive, self.name) )

        with self.counter.new_counter.request() as req:
          patience = random.uniform( MIN_PATIENCE, MAX_PATIENCE )
          yield req
          # print( "..............%7.4f patience %7.4f" % (self.env.now, patience) )
          wait = self.env.now - arrive
          env.process( self.processing(req, patience) )

          try:
              # We got to the counter
              print('%7.4f %s: Waited %6.3f' % (self.env.now, self.name, wait))
              tib = random.expovariate(1.0 / self.time_in_bank)
              yield self.env.timeout(tib)
              print( '%7.4f %s: Served by Counter %02d' % (self.env.now, self.name, self.counter.new_counter.count) )         
              # print( '**********************', len(self.counter.new_counter.queue) )
              # print( '**********************', len(self.counter.new_counter.users) )
              # print( '**********************', NO_OF_COUNTERS )
              self.count_cus -= 1
              
          except simpy.Interrupt:
              print( '%7.4f %s: About to give up after %6.3f' % (self.env.now, self.name, patience) )
              NO_OF_COUNTERS += 1
              print( '%7.4f Counter%02d: Opened' % (self.env.now, NO_OF_COUNTERS) )
              tib = random.expovariate(1.0 / self.time_in_bank)
              yield self.env.timeout(tib)             
              print( '%7.4f %s: Served by Counter %02d' % (self.env.now, self.name, self.counter.new_counter.count) )
              self.count_cus -= 1

          global COUNT_CUSTOMER
          COUNT_CUSTOMER = self.count_cus


    def processing (self, req, patience):     
        if ( len(self.counter.new_counter.users) > NO_OF_COUNTERS  and  NO_OF_COUNTERS <= 20 ):
            # yield self.env.timeout(patience)
            self.process.interrupt()

        elif ( len(self.counter.new_counter.users) == self.counter.new_counter.capacity ):
              #We reneged
              print('%7.4f %s: RENEGED' % (self.env.now, self.name))
              yield self.env.timeout(patience)
              self.count_cus -= 1


print('Bank renege')
random.seed(RANDOM_SEED)
env = simpy.Environment()
counter = Counter(env)
env.process( generate_customer( env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter ) )
env.run()

Bank renege
 0.0000 Counter01: Opened
 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 3.8595 Customer00: Served by Counter 01
10.2006 Customer01: Here I am
10.2006 Customer01: Waited  0.000
12.7265 Customer02: Here I am
12.7265 Customer02: Waited  0.000
12.7265 Customer02: About to give up after  1.174
12.7265 Counter02: Opened
13.0895 Customer02: Served by Counter 02
23.7507 Customer01: Served by Counter 01
34.9993 Customer03: Here I am
34.9993 Customer03: Waited  0.000
35.3221 Customer03: Served by Counter 01
37.4665 Customer04: Here I am
37.4665 Customer04: Waited  0.000
46.9145 Customer04: Served by Counter 01


In [3]:
#capacity 20 - preemption - printing j as counter number
import random
import simpy

RANDOM_SEED = 42
NEW_CUSTOMERS = 30  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience
no_of_counters = 0     #variable that keeps track of number of counters
count_customer = 0     #variable that keeps track of number of customers


class Counter(object):

    def __init__(self, env):
        global no_of_counters
        no_of_counters += 1
        self.env = env
        self.new_counter = simpy.PreemptiveResource( self.env, capacity = 20 )  #creates a new counter
        print( '%7.4f Counter%02d: Opened' % (self.env.now, no_of_counters) )


def source(env, number, interval, counter):
    global count_customer
    global no_of_counters

    for i in range(number):
        count_customer += 1
        if (count_customer > 5):
          no_of_counters += 1
          print("%7.4f: More than 5 customers waiting in line" % env.now)
          print( '%7.4f Counter%02d: Opened' % ( env.now, no_of_counters ) )
          count_customer = 1

        c = Customer(env, count_customer, 'Customer%02d' % i, counter, time_in_bank=12.0)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)


class Customer(object):

  def __init__(self, env, count_cus, name, counter, time_in_bank):
        self.env = env
        self.count_cus = count_cus
        self.name = name
        self.counter = counter
        self.time_in_bank = time_in_bank
        self.process = env.process(self.customer())


  def customer (self):
      global no_of_counters
      arrive = self.env.now      
      print( '%7.4f %s: Here I am' % (arrive, self.name) )

      with self.counter.new_counter.request() as req:
          patience = random.uniform( MIN_PATIENCE, MAX_PATIENCE )
          yield req
          wait = self.env.now - arrive

          if ( len(self.counter.new_counter.users) <= no_of_counters ):
              print('%7.4f %s: Waited %6.3f' % (self.env.now, self.name, wait))
              tib = random.expovariate(1.0 / self.time_in_bank)
              yield self.env.timeout(tib)

              j = 0
              for i in range(len(self.counter.new_counter.users)):
                if (self.counter.new_counter.users[i] == req):
                  j = i + 1

              # print("............................", j)
              print( '%7.4f %s: Served by Counter %02d' % (self.env.now, self.name, j) )
              self.count_cus -= 1           

          elif ( len(self.counter.new_counter.users) > no_of_counters  and  no_of_counters <= 20 ):
            
              print( '%7.4f %s: About to give up after %6.3f' % (self.env.now, self.name, patience) )
              no_of_counters += 1
              print( '%7.4f Counter%02d: Opened' % (self.env.now, no_of_counters) )
              tib = random.expovariate(1.0 / self.time_in_bank)
              yield self.env.timeout(tib)
              
              j = 0
              for i in range(len(self.counter.new_counter.users)):
                if (self.counter.new_counter.users[i] == req):
                  j = i + 1

              # print("............................", j)
              print( '%7.4f %s: Served by Counter %02d' % (self.env.now, self.name, j) )
              self.count_cus -= 1

          else:
              #We reneged
              print('%7.4f %s: RENEGED' % (self.env.now, self.name))
              yield self.env.timeout(patience)
              self.count_cus -= 1

      global count_customer
      count_customer = self.count_cus


# Setup and start the simulation
print('Bank renege')
random.seed(RANDOM_SEED)
env = simpy.Environment()
counter = Counter(env)
# Start processes and run
env.process( source( env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter ) )
env.run()

Bank renege
 0.0000 Counter01: Opened
 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 3.8595 Customer00: Served by Counter 01
10.2006 Customer01: Here I am
10.2006 Customer01: Waited  0.000
12.7265 Customer02: Here I am
12.7265 Customer02: About to give up after  1.174
12.7265 Counter02: Opened
19.3030 Customer02: Served by Counter 02
23.7507 Customer01: Served by Counter 01
34.9993 Customer03: Here I am
34.9993 Customer03: Waited  0.000
35.3018 Customer04: Here I am
35.3018 Customer04: Waited  0.000
35.5708 Customer05: Here I am
35.5708 Customer05: About to give up after  1.441
35.5708 Counter03: Opened
43.4441 Customer06: Here I am
43.4441 Customer06: About to give up after  1.013
43.4441 Counter04: Opened
43.4463 Customer03: Served by Counter 01
46.2485 Customer05: Served by Counter 02
47.8958 Customer04: Served by Counter 01
60.0215 Customer07: Here I am
60.0215 Customer07: Waited  0.000
62.0493 Customer07: Served by Counter 02
63.1117 Customer06: Served by Counter

In [9]:
#ANOTHER DISASTER!! - with capacity 1 - premption - if req in results condition - priority set as count_cus - 
#gives Interrupt(<simpy.resources.resource.Preempted object at 0x7f3e9ef1afd0>) error for more than 6 customer
import random
import simpy

RANDOM_SEED = 42
NEW_CUSTOMERS = 5  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience
no_of_counters = 0     #variable that keeps track of number of counters
count_customer = 0     #variable that keeps track of number of customers


class Counter(object):
    instances = {}
    def __init__(self, env):
        global no_of_counters
        no_of_counters += 1
        self.env = env
        self.new_counter = simpy.PreemptiveResource( self.env, capacity = 1 )  #creates a new counter
        self.counter_name = no_of_counters
        print( '%7.4f Counter%02d: Opened' % (self.env.now, self.counter_name) )
        Counter.instances["Counter %02d" % self.counter_name] = self 


def source(env, number, interval, counter):
    global count_customer

    for i in range(number):
        count_customer += 1
        if (count_customer > 5):
          print("%7.4f: More than 5 customers waiting in line" % env.now)
          counter = Counter(env)
          count_customer = 1

        c = customer(env, count_customer, 'Customer%02d' % i, counter, time_in_bank=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)


def customer (env, count_cus, name, counter, time_in_bank):
    arrive = env.now    
    print( '%7.4f %s: Here I am' % (arrive, name) )

    with counter.new_counter.request( priority = count_cus ) as req:
        patience = random.uniform( MIN_PATIENCE, MAX_PATIENCE )
        results = yield req | env.timeout(patience)
        wait = env.now - arrive

        if req in results:
            print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))
            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print( '%7.4f %s: Served by Counter %02d' % (env.now, name, counter.counter_name) )
            count_cus -= 1
            
        elif ( req not in results  and  no_of_counters <= 20 ):  
            print( '%7.4f %s: About to give up after %6.3f patience %7.4f' % (env.now, name, wait, patience) )
            counter = Counter(env)  
            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print( '%7.4f %s: Served by Counter %02d' % (env.now, name, counter.counter_name) )
            # print ("\nMonitor: ", counter.instances)
            count_cus -= 1

        else:
            # We reneged
            print('%7.4f %s: RENEGED after %6.3f' % (env.now, name, wait))
            yield env.timeout(patience)
            count_cus -= 1

    global count_customer
    count_customer = count_cus


# Setup and start the simulation
print('Bank renege')
random.seed(RANDOM_SEED)
env = simpy.Environment()
counter = Counter(env)
# Start processes and run
env.process( source( env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter ) )
env.run()

Bank renege
 0.0000 Counter01: Opened
 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 3.8595 Customer00: Served by Counter 01
10.2006 Customer01: Here I am
10.2006 Customer01: Waited  0.000
12.7265 Customer02: Here I am
13.9003 Customer02: About to give up after  1.174 patience  1.1739
13.9003 Counter02: Opened
20.4769 Customer02: Served by Counter 02
23.7507 Customer01: Served by Counter 01
34.9993 Customer03: Here I am
34.9993 Customer03: Waited  0.000
35.3018 Customer04: Here I am
36.6995 Customer04: About to give up after  1.398 patience  1.3977
36.6995 Counter03: Opened
43.4463 Customer03: Served by Counter 01
49.2934 Customer04: Served by Counter 03


In [11]:
#with capacity 1 - premption - if req in results condition - priority set as negative so smaller more priority - 
#gives Interrupt(<simpy.resources.resource.Preempted object at 0x7f3e9ef1afd0>) error for more than 6 customer
import random
import simpy

RANDOM_SEED = 42
NEW_CUSTOMERS = 5  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience
no_of_counters = 0     #variable that keeps track of number of counters
count_customer = 0     #variable that keeps track of number of customers
prio = 0


class Counter(object):
    instances = {}
    def __init__(self, env):
        global no_of_counters
        no_of_counters += 1
        self.env = env
        self.new_counter = simpy.PreemptiveResource( self.env, capacity = 1 )  #creates a new counter
        self.counter_name = no_of_counters
        print( '%7.4f Counter%02d: Opened' % (self.env.now, self.counter_name) )
        Counter.instances["Counter %02d" % self.counter_name] = self 


def source(env, number, interval, counter):
    global count_customer

    for i in range(number):
        count_customer += 1
        if (count_customer > 5):
          print("%7.4f: More than 5 customers waiting in line" % env.now)
          counter = Counter(env)
          prio -= 1
          print (".................priority", prio)
          count_customer = 1

        c = customer(env, count_customer, 'Customer%02d' % i, counter, time_in_bank=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)


def customer (env, count_cus, name, counter, time_in_bank):
    global prio
    arrive = env.now    
    print( '%7.4f %s: Here I am' % (arrive, name) )

    with counter.new_counter.request( priority = prio ) as req:
        patience = random.uniform( MIN_PATIENCE, MAX_PATIENCE )
        results = yield req | env.timeout(patience)
        wait = env.now - arrive

        if req in results:
            print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))
            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print( '%7.4f %s: Served by Counter %02d' % (env.now, name, counter.counter_name) )
            count_cus -= 1
            prio -= 1
            print (".................priority", prio)
            
        elif ( req not in results  and  no_of_counters <= 20 ):  
            print( '%7.4f %s: About to give up after %6.3f patience %7.4f' % (env.now, name, wait, patience) )
            counter = Counter(env)  
            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print( '%7.4f %s: Served by Counter %02d' % (env.now, name, counter.counter_name) )
            # print ("\nMonitor: ", counter.instances)
            count_cus -= 1
            prio -= 1
            print (".................priority", prio)

        else:
            # We reneged
            print('%7.4f %s: RENEGED after %6.3f' % (env.now, name, wait))
            yield env.timeout(patience)
            count_cus -= 1

    global count_customer
    count_customer = count_cus


# Setup and start the simulation
print('Bank renege')
random.seed(RANDOM_SEED)
env = simpy.Environment()
counter = Counter(env)
# Start processes and run
env.process( source( env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter ) )
env.run()

Bank renege
 0.0000 Counter01: Opened
 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 3.8595 Customer00: Served by Counter 01
.................priority -1
10.2006 Customer01: Here I am
10.2006 Customer01: Waited  0.000
12.7265 Customer02: Here I am
13.9003 Customer02: About to give up after  1.174 patience  1.1739
13.9003 Counter02: Opened
20.4769 Customer02: Served by Counter 02
.................priority -2
23.7507 Customer01: Served by Counter 01
.................priority -3
34.9993 Customer03: Here I am
34.9993 Customer03: Waited  0.000
35.3018 Customer04: Here I am
36.6995 Customer04: About to give up after  1.398 patience  1.3977
36.6995 Counter03: Opened
43.4463 Customer03: Served by Counter 01
.................priority -4
49.2934 Customer04: Served by Counter 03
.................priority -5


In [4]:
#capacity 1 - preemption -  if req in results: - RESTART PROCESS
import random
import simpy

RANDOM_SEED = 42
NEW_CUSTOMERS = 5  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience
RESTART_PROCESS = False
no_of_counters = 0     #variable that keeps track of number of counters
count_customer = 0     #variable that keeps track of number of customers


class Counter(object):
    instances = {}
    def __init__(self, env):
        global no_of_counters
        no_of_counters += 1
        self.env = env
        self.new_counter = simpy.PreemptiveResource( self.env, capacity = 1 )  #creates a new counter
        self.counter_name = no_of_counters
        print( '%7.4f Counter%02d: Opened' % (self.env.now, self.counter_name) )
        Counter.instances["Counter %02d" % self.counter_name] = self 

    # def process_req (self, prio):
    #     with self.new_counter.request( priority = prio ) as req:
    #         patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
    #         # Wait for the counter or abort at the end of our tether
    #         results = yield req | env.timeout(patience)
    #         if req in results:
    #           counter_num = prio
    #         else:
    #           counter_num = prio - 1        
    #     return counter_num, results


def source(env, number, interval, counter):
    global count_customer

    for i in range(number):
        count_customer += 1
        if (count_customer > 5):
          print("%7.4f: More than 5 customers waiting in line" % env.now)
          counter = Counter(env)
          count_customer = 1

        c = customer(env, count_customer, 'Customer%02d' % i, counter, time_in_bank=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)


def customer (env, count_cus, name, counter, time_in_bank):
    global RESTART_PROCESS
    arrive = env.now
    if (RESTART_PROCESS == False):       
        print( '%7.4f %s: Here I am' % (arrive, name) )

    with counter.new_counter.request() as req:
      patience = random.uniform( MIN_PATIENCE, MAX_PATIENCE )
      results = yield req | env.timeout(patience)
      wait = env.now - arrive

      if req in results:
          if (RESTART_PROCESS == False):
              # We got to the counter
              print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))

          tib = random.expovariate(1.0 / time_in_bank)
          yield env.timeout(tib)
          print( '%7.4f %s: Served by Counter %02d' % (env.now, name, counter.counter_name) )
          # print ("\nMonitor: ", counter.instances)
          # print ('cus count', count_cus)
          RESTART_PROCESS = False
          count_cus -= 1
          
      elif ( req not in results  and  no_of_counters <= 20 ):
          print( '%7.4f %s: About to give up after %6.3f patience %7.4f' % (env.now, name, wait, patience) )
          counter = Counter(env)
          RESTART_PROCESS = True
          env.process( customer(env, count_cus, name, counter, time_in_bank=12.0) )
          print (".................................................")
          # tib = random.expovariate(1.0 / time_in_bank)
          # yield env.timeout(tib)
          # print( '%7.4f %s: Served by Counter %02d' % (env.now, name, counter.counter_name) )
          # # print ("\nMonitor: ", counter.instances)
          # count_cus -= 1

      else:
          # We reneged
          print('%7.4f %s: RENEGED after %6.3f' % (env.now, name, wait))
          yield env.timeout(patience)
          count_cus -= 1

    global count_customer
    count_customer = count_cus


# Setup and start the simulation
print('Bank renege')
random.seed(RANDOM_SEED)
env = simpy.Environment()
counter = Counter(env)
# Start processes and run
env.process( source( env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter ) )
env.run()

Bank renege
 0.0000 Counter01: Opened
 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 3.8595 Customer00: Served by Counter 01
10.2006 Customer01: Here I am
10.2006 Customer01: Waited  0.000
12.7265 Customer02: Here I am
13.9003 Customer02: About to give up after  1.174 patience  1.1739
13.9003 Counter02: Opened
.................................................
14.2633 Customer02: Served by Counter 02
23.7507 Customer01: Served by Counter 01
34.9993 Customer03: Here I am
34.9993 Customer03: Waited  0.000
35.3221 Customer03: Served by Counter 01
37.4665 Customer04: Here I am
37.4665 Customer04: Waited  0.000
46.9145 Customer04: Served by Counter 01


In [5]:
#DISASTER!!! - capacity 1 - preempted - env.now == (arrive + patience) condition in if else
import random
import simpy

RANDOM_SEED = 42
NEW_CUSTOMERS = 5  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience
no_of_counters = 0     #variable that keeps track of number of counters
count_customer = 0     #variable that keeps track of number of customers


class Counter(object):
    instances = {}
    def __init__(self, env):
        global no_of_counters
        no_of_counters += 1
        self.env = env
        self.new_counter = simpy.PreemptiveResource( self.env, capacity = 1 )  #creates a new counter
        self.counter_name = no_of_counters
        print( '%7.4f Counter%02d: Opened' % (self.env.now, self.counter_name) )
        Counter.instances["Counter %02d" % self.counter_name] = self 


def source(env, number, interval, counter):
    global count_customer

    for i in range(number):
        count_customer += 1
        if (count_customer > 5):
          counter = Counter(env)
          count_customer = 1

        c = customer(env, count_customer, 'Customer%02d' % i, counter, time_in_bank=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)


def customer (env, count_cus, name, counter, time_in_bank):
    arrive = env.now    
    print( '%7.4f %s: Here I am' % (arrive, name) )

    with counter.new_counter.request( priority = count_cus, preempt = False ) as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        yield req
        wait = env.now - arrive

        if ( env.now == (arrive + patience)  and  no_of_counters <= 20 ):
            print( '%7.4f %s: About to give up after %6.3f patience %7.4f' % (env.now, name, wait, patience) )
            counter = Counter(env)  
            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print( '%7.4f %s: Served by Counter %02d' % (env.now, name, counter.counter_name) )
            # print ("\nMonitor: ", counter.instances)
            count_cus -= 1
            
        elif ( env.now == (arrive + patience)  and  no_of_counters > 20 ):
            # We reneged
            print('%7.4f %s: RENEGED after %6.3f' % (env.now, name, wait))
            yield env.timeout(patience)
            count_cus -= 1

        else:
            print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))
            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print( '%7.4f %s: Served by Counter %02d' % (env.now, name, counter.counter_name) )
            # print ("\nMonitor: ", counter.instances)
            count_cus -= 1

    global count_customer
    count_customer = count_cus


print('Bank renege')
random.seed(RANDOM_SEED)
env = simpy.Environment()
counter = Counter(env)
env.process( source( env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter ) )
env.run()

Bank renege
 0.0000 Counter01: Opened
 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 3.8595 Customer00: Served by Counter 01
10.2006 Customer01: Here I am
10.2006 Customer01: Waited  0.000
12.7265 Customer02: Here I am
23.7507 Customer01: Served by Counter 01
23.7507 Customer02: Waited 11.024
30.3272 Customer02: Served by Counter 01
34.9993 Customer03: Here I am
34.9993 Customer03: Waited  0.000
35.3018 Customer04: Here I am
43.4463 Customer03: Served by Counter 01
43.4463 Customer04: Waited  8.144
56.0402 Customer04: Served by Counter 01


In [22]:
# Customer is a method instead of a class here, capacity 20 - preemption - if ( len(counter.hold_resource.users) <= NO_OF_COUNTERS ) condition - seemingly works

import random
import simpy
RANDOM_SEED = 42  
NEW_CUSTOMERS = 5  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience
NO_OF_COUNTERS = 0     #variable that keeps track of number of counters
COUNT_CUSTOMER = 0     #variable that keeps track of number of customers

class Counter(object):
    # instances = {}
    def __init__(self, env):       
        global NO_OF_COUNTERS       
        NO_OF_COUNTERS += 1         
        self.env = env             
        self.hold_resource = simpy.PreemptiveResource( self.env, capacity = 20 ) 
        print( '%7.4f Counter%02d: Opened' % (self.env.now, NO_OF_COUNTERS) )     
        # Counter.instances["Counter %02d" % self.counter_name] = self 

def generate_customer(env, number, interval, counter):    
    global COUNT_CUSTOMER
    global NO_OF_COUNTERS

    for i in range(number):        
        COUNT_CUSTOMER += 1       
        if (COUNT_CUSTOMER > 5):    
          NO_OF_COUNTERS += 1       
          print("%7.4f: More than 5 customers waiting in line" % env.now)
          print( '%7.4f Counter%02d: Opened' % ( env.now, NO_OF_COUNTERS ) )                                                                               
          COUNT_CUSTOMER = 1    

        c = customer(env, COUNT_CUSTOMER, 'Customer%02d' % i, counter, time_in_bank=12.0) 
        env.process(c)
        t = random.expovariate(1.0 / interval)    
        yield env.timeout(t)             

def customer(env, count_cus, name, counter, time_in_bank):
    global NO_OF_COUNTERS
    arrive = env.now    
    print( '%7.4f %s: Here I am' % (arrive, name) )

    with counter.hold_resource.request(priority = count_cus) as req:
        patience = random.uniform( MIN_PATIENCE, MAX_PATIENCE )
        yield req
        # print( "..............%7.4f patience %7.4f" % (self.env.now, patience) )
        # env.process( self.processing(req, patience) )
        wait = env.now - arrive

        if ( len(counter.hold_resource.users) <= NO_OF_COUNTERS ):
            print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))
            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)

            # j = 0
            # for i in range(len(self.counter.hold_resource.users)):
            #   if (self.counter.hold_resource.users[i] == req):
            #     j = i + 1
            # print( '%7.4f %s: Served by Counter %02d' % (env.now, name, j) )

            print( '%7.4f %s: Served by Counter %02d' % (env.now, name, counter.hold_resource.count) )                        
            count_cus -= 1        

        elif ( len(counter.hold_resource.users) > NO_OF_COUNTERS  and  NO_OF_COUNTERS <= 20 ):
            print( '%7.4f %s: About to give up after %6.3f' % (env.now, name, patience) )
            yield env.timeout(patience)
            NO_OF_COUNTERS += 1
            print( '%7.4f Counter%02d: Opened' % (env.now, NO_OF_COUNTERS) )
            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)

            # j = 0
            # for i in range(len(self.counter.hold_resource.users)):
            #   if (self.counter.hold_resource.users[i] == req):
            #     j = i + 1
            # print( '%7.4f %s: Served by Counter %02d' % (env.now, name, j) )
            
            print( '%7.4f %s: Served by Counter %02d' % (env.now, name, counter.hold_resource.count) )
            count_cus -= 1

        else:
            #We reneged
            print('%7.4f %s: RENEGED' % (env.now, name))
            yield env.timeout(patience)
            count_cus -= 1

    global COUNT_CUSTOMER
    COUNT_CUSTOMER = count_cus

  # def processing (self, req, patience):
  #     if ( len(self.counter.hold_resource.users) > NO_OF_COUNTERS  and  NO_OF_COUNTERS <= 20 ):


print('Bank renege')
random.seed(RANDOM_SEED)
env = simpy.Environment()
counter = Counter(env)
env.process( generate_customer( env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter ) )
env.run()

Bank renege
 0.0000 Counter01: Opened
 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 3.8595 Customer00: Served by Counter 01
10.2006 Customer01: Here I am
10.2006 Customer01: Waited  0.000
12.7265 Customer02: Here I am
12.7265 Customer02: About to give up after  1.174
13.9003 Counter02: Opened
20.4769 Customer02: Served by Counter 02
23.7507 Customer01: Served by Counter 01
34.9993 Customer03: Here I am
34.9993 Customer03: Waited  0.000
35.3018 Customer04: Here I am
35.3018 Customer04: Waited  0.000
43.4463 Customer03: Served by Counter 02
47.8958 Customer04: Served by Counter 01
